<a href="https://colab.research.google.com/github/weathon/3d2smile/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""Untitled84.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1QyswmFpDOafdCi7qQgarEsNtNefHhs-D
"""

# https://www.tensorflow.org/tutorials/text/image_captioning

# # https://raw.githubusercontent.com/songlab-cal/tape/master/tape/tokenizers.py
# !wget https://raw.githubusercontent.com/songlab-cal/tape/master/tape/tokenizers.py

import re

def smiles_atom_tokenizer (smi):
    pattern =  "(\[[^\]]+]|Br?|Cl?|N|O|S|P|F|I|b|c|n|o|s|p|\(|\)|\.|=|#|-|\+|\\\\|\/|:|~|@|\?|>|\*|\$|\%[0-9]{2}|[0-9])"
    regex = re.compile(pattern)
    tokens = [token for token in regex.findall(smi)]
    return tokens

def duplicates(lst, item):
    return [i for i, x in enumerate(lst) if x == item]

def split_tokens(tokens_to_be_splitted, all_tokens):
    if ''.join(tokens_to_be_splitted) in all_tokens:
        indexes = duplicates(all_tokens, ''.join(tokens_to_be_splitted))
        for k, idx in enumerate(indexes):
            all_tokens[idx] = tokens_to_be_splitted[0]
            for i in range(1, len(tokens_to_be_splitted)):
                all_tokens.insert(idx+i, tokens_to_be_splitted[i])
            if k < len(indexes) - 1:
                indexes[k+1:] = list(map(lambda x: x+(len(tokens_to_be_splitted) - 1), indexes[k+1:]))
def add_symbol(pat):
    new_patt = pat[1:].split(',')
    ls_del = []
    if new_patt[0][0] == '0':
        ls_del.append(new_patt[0][0])
        new_patt[0] = new_patt[0][1:]

    while int(new_patt[0]) > int(new_patt[1]):
        ls_del.append(new_patt[0][0])
        new_patt[0] = new_patt[0][1:]
    result = ['.', ''.join(ls_del), '^', new_patt[0], ',', new_patt[1]]
    return result

def check_correct_tokenize(iupac,tokens):
    if ''.join(tokens).replace('^', '') == iupac:
        return True
    else:
        return False

def iupac_tokenizer(iupac):

    pattern = "\.\d+,\d+|[1-9]{2}\-[a-z]\]|[0-9]\-[a-z]\]|[1-9]{2}[a-z]|[1-9]{2}'[a-z]|[0-9]'[a-z]|[0-9][a-z]|\([0-9]\+\)|\([0-9]\-\)|" + \
              "[1-9]{2}|[0-9]|-|\s|\(|\)|S|R|E|Z|N|C|O|'|\"|;|λ|H|,|\.|\[[a-z]{2}\]|\[[a-z]\]|\[|\]|"

    alcane = "methane|methanoyl|methan|ethane|ethanoyl|ethan|propanoyl|propane|propan|propa|butane|butanoyl|butan|buta|pentane|" + \
             "pentanoyl|pentan|hexane|hexanoyl|hexan|heptane|heptanoyl|heptan|octane|octanoyl|octan|nonane|nonanoyl|" + \
             "nonan|decane|decanoyl|decan|icosane|icosan|cosane|cosan|contane|contan|"

    pristavka_name = "hydroxide|hydroxyl|hydroxy|hydrate|hydro|cyclo|spiro|iso|"
    pristavka_digit = "mono|un|bis|bi|dicta|di|tetraza|tetraz|tetra|tetr|pentaza|pentaz|penta|hexaza|" + \
                      "hexa|heptaza|hepta|octaza|octa|nonaza|nona|decaza|deca|kis|"

    prefix_alcane = "methylidene|methyl|ethyl|isopropyl|propyl|isobutyl|sec-butyl|tert-butyl|butyl|pentyl|hexyl|heptyl|octyl|"
    carbon = "meth|eth|prop|but|pent|hex|hept|oct|non|dec|icosa|icos|cosa|cos|icon|conta|cont|con|heni|hene|hen|hecta|hect|"

    prefix_all = "benzhydryl|benzoxaza|benzoxaz|benzoxa|benzox|benzo|benzyl|benz|phenacyl|phenanthro|phenyl|phenoxaza|phenoxaz|phenoxy|phenox|phenol|pheno|phen|acetyl|aceto|acet|" + \
                 "peroxy|oxido|oxino|oxalo|oxolo|oxocyclo|oxol|oxoc|oxon|oxo|oxy|pyrido|pyrimido|imidazo|naphtho|stiboryl|stibolo|"

    prefix_isotope = "protio|deuterio|tritio|"
    suffix_isotope = "protide|"
    prefix_galogen = "fluoro|fluoranyl|fluoridoyl|fluorido|chloro|chloranyl|chloridoyl|chlorido|bromo|bromanyl|bromidoyl|bromido|iodo|iodanyl|iodidoyl|iodanuidyl|iodido|"
    suffix_galogen = "fluoride|chloride|chloridic|perchloric|bromide|iodide|iodane|hypoiodous|hypochlorous|"
    prefix_chalcogen = "phosphonato|phosphoroso|phosphonia|phosphoryl|phosphanyl|arsono|arsanyl|stiba|"
    suffix_chalcogen = "phosphanium|phosphate|phosphite|phosphane|phosphanide|phosphonamidic|phosphonous|phosphinous|phosphinite|phosphono|arsonic|stibane|"
    prefix_metal = "alumanyl|gallanyl|stannyl|plumbyl|"
    suffix_metal = "chromium|stannane|gallane|alumane|aluminane|aluminan|"
    prefix_non_metal = "tellanyl|germanyl|germyl|"
    suffix_non_metal = "germane|germa|"

    prefix_sulfur = "sulfanylidene|sulfinamoyl|sulfonimidoyl|sulfinimidoyl|sulfamoyl|sulfonyl|sulfanyl|sulfinyl|sulfinato|sulfenato|" + \
                    "sulfonato|sulfonio|sulfino|sulfono|sulfido|"
    suffix_sulfur = "sulfonamide|sulfinamide|sulfonamido|sulfonic|sulfamic|sulfinic|sulfuric|thial|thione|thiol|" + \
                    "sulfonate|sulfite|sulfate|sulfide|sulfinate|sulfanium|sulfamate|sulfane|sulfo|"

    prefix_nitrogen = "hydrazono|hydrazino|nitroso|nitrous|nitro|formamido|amino|amido|imino|imido|anilino|anilin|thiocyanato|cyanato|cyano|azido|azanidyl|azanyl|" + \
                      "azanide|azanida|azonia|azonio|amidino|nitramido|diazo|"
    suffix_nitrogen = "ammonium|hydrazide|hydrazine|hydrazin|amine|imine|oxamide|nitramide|formamide|cyanamide|amide|imide|amidine|isocyanide|azanium|" + \
                      "thiocyanate|cyanate|cyanic|cyanatidoyl|cyanide|nitrile|nitrite|hydrazonate|"

    suffix_carbon = "carbonitrile|carboxamide|carbamimidothioate|carbodithioate|carbohydrazonate|carbonimidoyl|carboximidoyl|" + \
                    "carbamimidoyl|carbamimidate|carbamimid|carbaldehyde|carbamate|carbothioyl|carboximidothioate|carbonate|" + \
                    "carboximidamide|carboximidate|carbamic|carbonochloridate|carbothialdehyde|carbothioate|carbothioic|carbono|carbon|carbo|" + \
                    "formate|formic|"
    prefix_carbon = "carboxylate|carboxylato|carboxylic|carboxy|halocarbonyl|carbamoyl|carbonyl|carbamo|thioformyl|formyl|"

    silicon = "silanide|silane|silole|silanyl|silyloxy|silylo|silyl|sila|"
    boron = "boranyl|boranuide|boronamidic|boranuida|boranide|borinic|borate|borane|boran|borono|boron|bora|"
    selenium = "selanyl|seleno|"

    suffix_all = "ane|ano|an|ene|enoxy|eno|en|yne|yn|yl|peroxol|peroxo|" + \
                 "terephthalate|terephthalic|phthalic|phthalate|oxide|oate|ol|oic|ic|al|ate|ium|one|"

    carbon_trivial = "naphthalen|naphthal|inden|adamant|fluoren|thiourea|urea|anthracen|acenaphthylen|" + \
                     "carbohydrazide|annulen|aniline|acetaldehyde|benzaldehyde|formaldehyde|phthalaldehyde|acephenanthrylen|" + \
                     "phenanthren|chrysen|carbanid|chloroform|fulleren|cumen|formonitril|fluoranthen|terephthalaldehyde|azulen|picen|" + \
                     "pyren|pleiaden|coronen|tetracen|pentacen|perylen|pentalen|heptalen|cuban|hexacen|oxanthren|ovalen|aceanthrylen|"

    heterocycles = "indolizin|arsindol|indol|furan|furo|piperazin|pyrrolidin|pyrrolizin|thiophen|thiolo|imidazolidin|imidazol|pyrimidin|pyridin|" + \
                    "piperidin|morpholin|pyrazol|pyridazin|oxocinnolin|cinnolin|pyrrol|thiochromen|oxochromen|chromen|quinazolin|phthalazin|quinoxalin|carbazol|xanthen|pyrazin|purin|" + \
                    "indazol|naphthyridin|quinolizin|guanidin|pyranthren|pyran|thianthren|thian|acridin|acrido|yohimban|porphyrin|pteridin|tetramin|pentamin|" + \
                    "borinin|borino|boriran|borolan|borol|borinan|phenanthridin|quinolin|perimidin|corrin|phenanthrolin|phosphinolin|indacen|silonin|borepin|"

    prefix_heterocycles = "thiaz|oxaza|oxaz|oxan|oxa|ox|aza|az|thia|thioc|thion|thio|thi|telluro|phospha|phosph|selen|bor|sil|alum|ars|germ|tellur|imid|"

    suffix_heterocycles = "ir|et|olo|ol|ino|in|ep|oc|on|ec|"
    saturated_unsatured = "idine|idene|idin|ane|an|ine|in|id|e|"
    pristavka_exception = "do|trisodium|tris|triacetyl|triamine|triaza|triaz|tria|trityl|tri|o"

    type_ = "acid|ether|"
    element = "hydrogen|helium|lithium|beryllium|nitrogen|oxygen|fluorine|neon|sodium|magnesium|aluminum|silicon|" + \
              "phosphorus|sulfur|chlorine|argon|potassium|calcium|scandium|titanium|vanadium|chromium|manganese|iron|" + \
              "cobalt|nickel|copper|zinc|gallium|germanium|arsenic|selenium|bromine|krypton|rubidium|yttrium|zirconium|" + \
              "niobium|molybdenum|technetium|ruthenium|rhodium|palladium|silver|cadmium|indium|antimony|tellurium|iodine|" + \
              "xenon|cesium|barium|lanthanum|cerium|praseodymium|neodymium|latinum|promethium|samarium|europium|gadolinium|" + \
              "terbium|dysprosium|holmium|erbium|thulium|ytterbium|lutetium|hafnium|tantalum|tungsten|rhenium|osmium|" + \
              "iridium|platinum|gold|aurum|mercury|thallium|lead|bismuth|polonium|astatine|radon|francium|radium|actinium|" + \
              "thorium|protactinium|uranium|neptunium|plutonium|americium|curium|berkelium|einsteinium|fermium|californium|" + \
              "mendelevium|nobelium|lawrencium|rutherfordium|dubnium|seaborgium|bohrium|hassium|meitnerium|tin|"

    other_ions = "perchlorate|perbromate|periodate|hypofluorite|hypochlorite|hypobromite|hypoiodite|nitrate|silicate|hydride|"

    regex = re.compile(pattern + heterocycles + carbon_trivial + type_ + element + prefix_isotope + other_ions + alcane + pristavka_digit + pristavka_name + prefix_alcane + \
                       carbon + silicon + prefix_nitrogen + prefix_sulfur + prefix_carbon + prefix_metal + prefix_non_metal + prefix_all + prefix_galogen + prefix_chalcogen + \
                       suffix_carbon + suffix_nitrogen + suffix_sulfur + suffix_galogen + suffix_chalcogen + suffix_metal + suffix_non_metal + suffix_all + suffix_heterocycles + \
                       suffix_isotope + boron + selenium  + prefix_heterocycles + saturated_unsatured + pristavka_exception)
    tokens = [token for token in regex.findall(iupac)]

    split_tokens(['meth', 'ane'], tokens)
    split_tokens(['meth', 'an'], tokens)
    split_tokens(['eth', 'ane'], tokens)
    split_tokens(['eth', 'an'], tokens)
    split_tokens(['prop', 'ane'], tokens)
    split_tokens(['prop', 'an'], tokens)
    split_tokens(['but', 'ane'], tokens)
    split_tokens(['but', 'an'], tokens)
    split_tokens(['pent', 'ane'], tokens)
    split_tokens(['pent', 'an'], tokens)
    split_tokens(['hex', 'ane'], tokens)
    split_tokens(['hex', 'an'], tokens)
    split_tokens(['hept', 'ane'], tokens)
    split_tokens(['hept', 'an'], tokens)
    split_tokens(['oct', 'ane'], tokens)
    split_tokens(['oct', 'an'], tokens)
    split_tokens(['non', 'ane'], tokens)
    split_tokens(['non', 'an'], tokens)
    split_tokens(['dec', 'ane'], tokens)
    split_tokens(['dec', 'an'], tokens)
    split_tokens(['cos', 'ane'], tokens)
    split_tokens(['cos', 'an'], tokens)
    split_tokens(['cont', 'ane'], tokens)
    split_tokens(['cont', 'an'], tokens)
    split_tokens(['icos', 'ane'], tokens)
    split_tokens(['icos', 'an'], tokens)

    split_tokens(['thi', 'az'], tokens)
    split_tokens(['thi', 'oc'], tokens)
    split_tokens(['thi', 'on'], tokens)
    split_tokens(['benz', 'ox'], tokens)
    split_tokens(['benz', 'oxa'], tokens)
    split_tokens(['benz', 'ox', 'az'], tokens)
    split_tokens(['benz', 'ox', 'aza'], tokens)
    split_tokens(['phen', 'ox'], tokens)
    split_tokens(['phen', 'oxy'], tokens)
    split_tokens(['phen', 'oxa'], tokens)
    split_tokens(['phen', 'ox', 'az'], tokens)
    split_tokens(['phen', 'ox', 'aza'], tokens)
    split_tokens(['phen', 'ol'], tokens)
    split_tokens(['en', 'oxy'], tokens)
    split_tokens(['ox', 'az'], tokens)
    split_tokens(['ox', 'aza'], tokens)
    split_tokens(['tri', 'az'], tokens)
    split_tokens(['tri', 'amine'], tokens)
    split_tokens(['tri', 'acetyl'], tokens)
    split_tokens(['ox', 'ol'], tokens)
    split_tokens(['ox', 'olo'], tokens)
    split_tokens(['ox', 'an'], tokens)
    split_tokens(['ox', 'oc'], tokens)
    split_tokens(['ox', 'on'], tokens)
    split_tokens(['tri', 'az'], tokens)
    split_tokens(['tri', 'aza'], tokens)
    split_tokens(['tri', 'sodium'], tokens)
    split_tokens(['tetr', 'az'], tokens)
    split_tokens(['tetr', 'aza'], tokens)
    split_tokens(['pent', 'az'], tokens)
    split_tokens(['pent', 'aza'], tokens)
    split_tokens(['hex', 'aza'], tokens)
    split_tokens(['hept', 'aza'], tokens)
    split_tokens(['oct', 'aza'], tokens)
    split_tokens(['non', 'aza'], tokens)
    split_tokens(['dec', 'aza'], tokens)
    split_tokens(['oxo', 'chromen'], tokens)
    split_tokens(['oxo', 'cinnolin'], tokens)
    split_tokens(['oxo', 'cyclo'], tokens)
    split_tokens(['thio', 'chromen'], tokens)
    split_tokens(['thio', 'cyanato'], tokens)

    if (len(re.findall(re.compile('[0-9]{2}\-[a-z]\]'), ''.join(tokens))) > 0):
        for i, tok in enumerate(tokens):
            if re.findall(re.compile('[0-9]{2}\-[a-z]\]'), tok):
                tokens[i] = tok[:2]
                tokens.insert(i+1,tok[2])
                tokens.insert(i+2,tok[3])
                tokens.insert(i+3,tok[4])

    if (len(re.findall(re.compile('[0-9]\-[a-z]\]'), ''.join(tokens))) > 0):
        for i, tok in enumerate(tokens):
            if re.findall(re.compile('[0-9]\-[a-z]\]'), tok):
                tokens[i] = tok[:1]
                tokens.insert(i+1,tok[1])
                tokens.insert(i+2,tok[2])
                tokens.insert(i+3,tok[3])

    if (len(re.findall(re.compile('\[[a-z]{2}\]'), ''.join(tokens))) > 0):
        for i, tok in enumerate(tokens):
            if re.findall(re.compile('\[[a-z]{2}\]'), tok):
                tokens[i] = tok[0]
                tokens.insert(i+1,tok[1])
                tokens.insert(i+2,tok[2])
                tokens.insert(i+3,tok[3])

    if (len(re.findall(re.compile('\[[a-z]\]'), ''.join(tokens))) > 0):
        for i, tok in enumerate(tokens):
            if re.findall(re.compile('\[[a-z]\]'), tok):
                tokens[i] = tok[0]
                tokens.insert(i+1,tok[1])
                tokens.insert(i+2,tok[2])

    if (len(re.findall(re.compile("[0-9]{2}'[a-z]"), ''.join(tokens))) > 0):
        for i, tok in enumerate(tokens):
            if re.findall(re.compile("[0-9]{2}'[a-z]"), tok):
                tokens[i] = tok[:2]
                tokens.insert(i+1,tok[2])
                tokens.insert(i+2,tok[3])

    if (len(re.findall(re.compile("[0-9]'[a-z]"), ''.join(tokens))) > 0):
        for i, tok in enumerate(tokens):
            if re.findall(re.compile("[0-9]'[a-z]"), tok):
                tokens[i] = tok[0]
                tokens.insert(i+1,tok[1])
                tokens.insert(i+2,tok[2])

    if (len(re.findall(re.compile("[0-9]{2}[a-z]"), ''.join(tokens))) > 0):
        for i, tok in enumerate(tokens):
            if re.findall(re.compile("[0-9]{2}[a-z]"), tok):
                tokens[i] = tok[:2]
                tokens.insert(i+1,tok[2])

    if (len(re.findall(re.compile("[0-9][a-z]"), ''.join(tokens))) > 0):
        for i, tok in enumerate(tokens):
            if re.findall(re.compile("[0-9][a-z]"), tok):
                tokens[i] = tok[0]
                tokens.insert(i+1,tok[1])

    if (len(re.findall(re.compile("\.\d+,\d+"), ''.join(tokens))) > 0):
        for i, tok in enumerate(tokens):
            if re.findall(re.compile("\.\d+,\d+"), tok):
                result = add_symbol(tok)
                tokens[i] = result[0]
                for k, res in enumerate(result[1:], start=1):
                    tokens.insert(i+k,res)

    if check_correct_tokenize(iupac, tokens) == True:
        return tokens
    else:
        return None

"""## Model Definition"""

import tensorflow as tf
import numpy as np
from PIL import Image
import pylab

CHANNELS = [64, 64, 128, 256, 512]
RES_NUMBER = 4
EMBEDING_DIM =1024
HEADS = 16
DECODER_NUM = 1

class ResNetBlock(tf.keras.layers.Layer):
  def __init__(self, channel1, channel2, name):
    super().__init__(name=name)
    self.Conv2D = tf.keras.Sequential([
        tf.keras.layers.Conv2D(channel1, (3,3), padding="same", strides = 2),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.ReLU(),
        tf.keras.layers.Conv2D(channel2, (3,3), padding="same"),
        tf.keras.layers.BatchNormalization()
    ])
    self.project =  tf.keras.layers.Conv2D(channel2, (1,1), name="image_projection", strides = 2)
    self.relu = tf.keras.layers.ReLU()

  def call(self, images):
    x = self.Conv2D(images)
    images = self.project(images)
    return self.relu(x+images)

class PosEncoding(tf.keras.layers.Layer):
  def __init__(self, max_length):
    super().__init__()
    self.max_length = max_length
    self.Embeding =  tf.keras.layers.Embedding(max_length, EMBEDING_DIM, input_length=max_length)

  def call(self, x):
    tags = tf.range(self.max_length)
    return self.Embeding(tags[tf.newaxis,:]) + x


class ImageEncoder(tf.keras.layers.Layer):
  def __init__(self):
    super().__init__()
    self.conv2D_7 = tf.keras.layers.Conv2D(CHANNELS[0], (7, 7), padding="same", strides = 2)
    self.pooling = tf.keras.layers.MaxPooling2D(pool_size=(3,3), strides=2)
    self.ResBlocks = []
    for i in range(RES_NUMBER):
      self.ResBlocks.append(ResNetBlock(CHANNELS[i+1], CHANNELS[i+1]*2, name=f"resblock_{i}"))
    self.flatten = tf.keras.layers.Reshape((49,1024))
    self.img_pos_encoding = PosEncoding(49)
    self.attention = tf.keras.layers.Attention()

  def call(self, images):
    images = self.conv2D_7(images)
    images = self.pooling(images)
    for block in self.ResBlocks:
      images = block(images)
    images = self.flatten(images)
    images = self.img_pos_encoding(images)
    return images #self.attention([images, images, images])

  # def model(self):
  #     x = tf.keras.layers.Input(shape=(400, 400, 3))
  #     return tf.keras.Model(inputs=[x], outputs=self.call(x))
  # https://stackoverflow.com/questions/55235212/model-summary-cant-print-output-shape-while-using-subclass-model

class AddNorm(tf.keras.layers.Layer):
  def __init__(self):
    super().__init__()
    self.norm = tf.keras.layers.LayerNormalization()

  def call(self, x, y):
    return self.norm(x+y)


class TextEncoder(tf.keras.layers.Layer):
  def __init__(self, num_token, max_length, first_layer, name=""):
    if name == "":
      super().__init__()
    else:
      super().__init__(name=name)
    self.first_layer = first_layer
    self.num_token, self.max_length = num_token, max_length
    self.pos_encoding = PosEncoding(max_length) #added one above already
    self.Embeding =  tf.keras.layers.Embedding(num_token, EMBEDING_DIM, input_length=max_length)
    self.attention = tf.keras.layers.MultiHeadAttention(HEADS, 1024, dropout=0.3)
    self.add_and_norm = AddNorm()

  def call(self, x):
    if self.first_layer:
      x = self.Embeding(x)
      x = self.pos_encoding(x)
    attention_score = self.attention(x, x, use_causal_mask=1)
    return self.add_and_norm(attention_score, x)

class CrossAttention(tf.keras.layers.Layer):
  def __init__(self, num_token):
    super().__init__()
    self.MultiHeadAttention = tf.keras.layers.MultiHeadAttention(HEADS, 1024, dropout=0.3)

  def call(self, image_features, texts):
    return self.MultiHeadAttention(value=image_features, query=texts)

class Decoder(tf.keras.layers.Layer):
  def __init__(self, num_token, max_length):
    super().__init__()
    # self.TextEncoder = TextEncoder(num_token, max_length)

    # input = tf.keras.layers.Input(shape=(400, 400, 3))
    self.ImageEncoder = ImageEncoder()
    #  tf.keras.Model(inputs=[input], outputs=ImageEncoder().call(input))

    # # input2 = tf.keras.layers.Input(shape=(100, 500))
    # #tf.keras.Model(inputs=[input2], outputs=TextEncoder(100, 500).call(input2))
    # input2 = tf.keras.layers.Input(shape=(500))
    # self.TextEncoder = tf.keras.Model(inputs=[input2], outputs=TextEncoder(100, 500).call(input2))

    self.CrossAttentions = []
    self.add_and_norm1s = []
    self.add_and_norm2s = []
    self.MLPs = []
    self.TextEncoders = []


    for i in range(DECODER_NUM):
      self.TextEncoders.append(TextEncoder(num_token, max_length, i==0))
      self.CrossAttentions.append(CrossAttention(num_token))
      self.add_and_norm1s.append(AddNorm())
      self.add_and_norm2s.append(AddNorm())
      self.MLPs.append(tf.keras.Sequential([
          tf.keras.layers.Dense(2048, activation="relu"),
          tf.keras.layers.Dense(1024, activation="relu"),
          tf.keras.layers.Dropout(0.1)], name=f"MLP_{i}"))

    self.DenseWithSoftmax = tf.keras.Sequential(
        [tf.keras.layers.Dense(num_token),
        tf.keras.layers.Dense(num_token, activation="softmax")], name="linear_and_softmax")

  def call(self, images, texts):
    images = self.ImageEncoder(images)
    for i in range(DECODER_NUM):
      texts = self.TextEncoders[i](texts)
      features = self.CrossAttentions[i](images, texts)
      features = self.add_and_norm1s[i](features, texts)
      texts = self.add_and_norm2s[i](self.MLPs[i](features), features)
      # text = self.add_and_norm2s[i](self.MLPs[i](features), features)
    return self.DenseWithSoftmax(texts)

model = Decoder(100, 500)
input1 = tf.keras.layers.Input(shape=(400, 400, 3))
input2 = tf.keras.layers.Input(shape=(500)) #input is NOT onehot
model =  tf.keras.Model(inputs=[input1, input2], outputs=model.call(input1, input2))
model.summary(expand_nested=1)
#tf.keras.utils.plot_model(model)

#tf.keras.utils.plot_model(model, expand_nested=1, rankdir="LR") #https://stackoverflow.com/questions/62337528/how-to-plot-machine-learning-model-horizontally

#itf.keras.utils.plot_model(model) #https://stackoverflow.com/questions/62337528/how-to-plot-machine-learning-model-horizontally

"""## Data Pre-processing

### Text
"""

import os
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False
"""
if IN_COLAB:
  if not "80k" in "".join(os.listdir("/home/wg25r/colab")):
    os.system("cp drive/MyDrive/80k.zip .")
    os.system("cp drive/MyDrive/80k.csv .")
    os.system("unzip 80k.zip")
else:
    if not "80k" in "".join(os.listdir("/home/wg25r/colab")):
      os.system("wget file.weasoft.com/80k.zip")
      os.system("wget http://file.weasoft.com/80k.csv")
      os.system("unzip 80k.zip")
"""
import os
ids = [i.split("_")[0] for i in os.listdir("/arc/project/st-dushan20-1/rendered/rendered")]

with open("ids.txt", "w") as f:
    f.write("\n".join(list(set(ids))))

import pandas as pd

csv = pd.read_csv("80k.csv")

csv.columns
#kk fucking eifaxian iupacname

def valid_iupac(i):
  try:
    return (iupac_tokenizer(i)!=None)
  except:
    return (False)

cids = csv["cid"]

Ys = {}
invalid_cid = []
for i in cids.values:
  try:
    tmp = iupac_tokenizer(csv[csv["cid"] == i]["iupacname"].values[0])
    if not tmp == None:
      Ys[i] = tmp
    else:
      invalid_cid.append(i)

  except:
    invalid_cid.append(i)

len(invalid_cid)

Ys_in = {}
Ys_out = {}
tokens = []
max_length = 0
for i in Ys.keys():
  # Ys_in[i] = "😘" + Ys[i]
  # Ys_out[i] = Ys[i] + "😢"
  tokens.extend(Ys[i])
  max_length = max(max_length, len(Ys[i]))


tokens = ["🍜","😘","😢"]+list(set(tokens))

def str_to_tokens(a: str):
  return [tokens.index(i) for i in a]

for i in Ys.keys():
  t = str_to_tokens(Ys[i])
  # Ys_in[i] = [1] + t + ([0] * (max_length-len(t)))
  Ys_in[i] =  ([0] * (max_length-len(t))) + [1] + t
  Ys_out[i] = ([0] * (max_length-len(t))) + t + [2]
max_length += 1

len(Ys_in[list(Ys.keys())[1]]), max_length

"""### Image"""

# No processing at this point

invalid_cid = set(invalid_cid)

"""## Pipeline"""

# https://medium.com/analytics-vidhya/write-your-own-custom-data-generator-for-tensorflow-keras-1252b64e41c3
class CustomDataGen(tf.keras.utils.Sequence):
    def __init__(self, batch_size, training):
      self.batch_size = batch_size
      self.dir_ = os.listdir("/arc/project/st-dushan20-1/rendered/rendered")
      self.dir = []
      for i in self.dir_:
        if not int(i.split("_")[0]) in invalid_cid:
          self.dir.append(i)
      np.random.shuffle(self.dir)
      self.ids = np.array(set([i.split("_")[0] for i in self.dir]))
      self.offset = self.dir[:-2048] if training else self.dir[2048:]

    def on_epoch_end(self):
        np.random.shuffle(self.dir)

    def __getitem__(self, index_):
        start_index = index_ * self.batch_size
        Xs_img = []
        Xs_text = []
        y = [] #This is slow, rewrite later
        for _ in range(self.batch_size):
          index = start_index + _
          try:
            id = int(self.dir[index].split("_")[0])
          except:
            break
          index = start_index + _
          img = np.array(Image.open(f"/arc/project/st-dushan20-1/rendered/rendered/{self.dir[index]}").rotate(np.random.uniform(0,360), expand = 1).resize((400,400)), dtype="float32")
          np.random.uniform(size=img.shape)*20
          img += noise
          Xs_img.append(img)
          Xs_text.append(Ys_in[id])
          y.append(Ys_out[id])
        return [np.array(Xs_img), np.array(Xs_text)], np.array(y)

    def __len__(self):
        return len(self.dir)//self.batch_size//30

img = np.array(Image.open(f"/arc/project/st-dushan20-1/rendered/rendered/10011914_0.jpg").rotate(np.random.uniform(0,180)), dtype="float32")
noise = np.random.uniform(size=img.shape)*20
#pylab.imshow(np.array(noise+img, dtype="uint8"))

for i in os.listdir("/arc/project/st-dushan20-1/rendered/rendered"):
  if not "_" in i:
    print("P")

gen = CustomDataGen(256, 1)
gen.__getitem__(1)

"""## Training"""

model = Decoder(len(tokens), max_length)
#input1 = tf.keras.layers.Input(shape=(400, 400, 3))
#input2 = tf.keras.layers.Input(shape=(max_length)) #input is NOT onehot
#model =  tf.keras.Model(inputs=[input1, input2], outputs=model.call(input1, input2))
#model.summary(expand_nested=1)

def predict(model, img_id):
    seq = np.array([1]+[0]*(max_length-1))
    pred = model([np.array([np.array(Image.open(f"/arc/project/st-dushan20-1/rendered/rendered/{img_id}_0.jpg"), dtype="float32")]), np.array([seq])])[0]
    return "".join([tokens[tf.argmax(i)] for i in pred])

img_id = 10011914
print(predict(model, 10011914))
#pylab.imshow(Image.open(f"/arc/project/st-dushan20-1/rendered/rendered/{10011914}_1.jpg"))

# https://www.tensorflow.org/tutorials/text/image_captioning
# https://keras.io/api/losses/probabilistic_losses/#categorical_crossentropy-function
# https://arxiv.org/pdf/1706.03762.pdf
# https://towardsdatascience.com/what-is-label-smoothing-108debd7ef06
cce = tf.keras.losses.CategoricalCrossentropy(reduction=tf.keras.losses.Reduction.NONE, label_smoothing=0.1)

token_count = len(tokens)
def masked_loss(labels, preds):
  loss = cce(tf.one_hot(labels, token_count), preds)
  mask = tf.cast(labels != 0, dtype=loss.dtype)
  return tf.math.reduce_sum(loss*mask)/tf.math.reduce_sum(mask)

def masked_acc(labels, preds):
  pred = tf.cast(tf.argmax(preds, axis=-1), tf.float32)
  mask = tf.cast(labels != 0, tf.float32)
  labels = tf.cast(labels, tf.float32)
  match_case = tf.cast(labels == pred, tf.float32)
  return tf.math.reduce_sum(match_case*mask)/tf.math.reduce_sum(mask)

model.compile(optimizer=tf.keras.optimizers.Adam(),
          loss=masked_loss,metrics=[masked_acc])

class my_callback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    old_lr = self.model.optimizer.lr.read_value()
    new_lr = old_lr * 0.95
    print("\nepoch: {}   lr: {} -> {}".format(epoch, old_lr, new_lr))
    print(predict(self.model, 10011914))
    self.model.optimizer.lr.assign(new_lr)

  # def on_batch_end(self, batch, logs=None):
  #   old_lr = self.model.optimizer.lr.read_value()
  #   new_lr = old_lr * 0.999995
  #   self.model.optimizer.lr.assign(new_lr)
import datetime
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


train = CustomDataGen(64, 1)
val = CustomDataGen(64, 0)
model.fit(train, validation_data=val, epochs=5, callbacks=[my_callback(), tensorboard_callback])